In [4]:
from utils.dataset_utils import load_data, check_img, prepare_data
from utils.training_utils import build_model
from utils.format_utils import print_separator

In [5]:
import os
import warnings
import random
import numpy as np
from datetime import datetime
import pandas as pd
from matplotlib import pyplot as plt

from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

import tensorflow as tf

from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from keras.utils import to_categorical
from keras.applications.efficientnet_v2 import EfficientNetV2S
from keras.utils import to_categorical

In [6]:
np.set_printoptions(precision=2)  

In [7]:
DATA_SET_NAME = 'Kinect_clean'
ARCHITECTURE = 'efficien_net_v2'
IMG_MODE = 'depth'
DATE = datetime.now().strftime('%Y-%m-%d-%H-%M-%S')

OUTPUT_DIR_PATH = os.path.join(
    'results',
    DATA_SET_NAME,
    ARCHITECTURE,
    IMG_MODE,
    DATE
)
os.makedirs(OUTPUT_DIR_PATH, exist_ok=True)

EPOCHS = 1000
BATCH_SIZE = 32

data_path = './Data/Kinect-clean'
model_path = './models/Test_300_RGBD.h5'

In [8]:
emotions_dict = {
    "Colere": 0,
    "Degout": 1,
    "Joie": 2,
    "Neutralite": 3,
    "Peur": 4,
    "Surprise": 5,
    "Tristesse": 6
}

emotions_dict_inv = {v:k for k, v in emotions_dict.items()}

In [10]:
imgs_rgb, imgs_depth, labels = load_data(data_path)
k_fold, le = prepare_data(imgs_depth, labels)

In [11]:
labels = ['Colere', 'Degout', 'Joie', 'Neutre', 'Peur', 'Surprise', 'Tristesse']
cols = ['Colere', 'Degout', 'Joie', 'Neutre', 'Peur', 'Surprise', 'Tristesse', 'Fold_mean_accuracy', 'Fold_std']

In [ ]:
fold_accuracies, all_probabilities = [], []
accuracy_per_fold = []

In [12]:
for fold, (X_train, y_train, X_test, y_test) in enumerate(k_fold):
    
    os.makedirs(os.path.join(OUTPUT_DIR_PATH, f'fold_{fold+1}'), exist_ok=True)
    os.makedirs(os.path.join(OUTPUT_DIR_PATH, f'fold_{fold+1}', 'false_predict'), exist_ok=True)
    os.makedirs(os.path.join(OUTPUT_DIR_PATH, f'fold_{fold+1}', 'weights'), exist_ok=True)
    
    print(f"Fold {fold + 1}: Train Data Shape: {X_train.shape}, Train Labels Shape: {y_train.shape}")
    print(f"Fold {fold + 1}: Test Data Shape: {X_test.shape}, Test Labels Shape: {y_test.shape}\n")
    
    
    
    
    best_test_accuracy = 0
    saved_weight_files = []
    for i in range(3):
        checkpoint_path = os.path.join(OUTPUT_DIR_PATH, f'fold_{fold+1}', 'weights', f'best_model_{i+1}.h5')
        saved_weight_files.append(checkpoint_path)
        checkpoint = ModelCheckpoint(checkpoint_path, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
        earlystopping = EarlyStopping(monitor='val_accuracy', patience=15, verbose=1, mode='auto', restore_best_weights=True)
        rlrop = ReduceLROnPlateau(monitor='val_accuracy', mode='max', patience=5, factor=0.5, min_lr=1e-6, verbose=1)
        
        print(f'### Essai : {i+1} / 3 ###')
        model = build_model(model_path)

        hist = model.fit(
            X_train,
            y_train,
            batch_size=BATCH_SIZE,
            validation_data=(X_test, y_test),
            epochs=EPOCHS,
            callbacks=[checkpoint, earlystopping, rlrop]
        )
        
        test_loss, test_acc = model.evaluate(X_test, y_test, batch_size=BATCH_SIZE)
    
        print(f"Test Accuracy for Fold {fold + 1}, try  {i + 1} : {test_acc:.2f}") 
        
        if test_acc > best_test_accuracy:
            best_test_accuracy = test_acc
            best_hist = hist
            best_model = model
            best_checkpoint = checkpoint_path
        
    for weight_file in saved_weight_files:
        if weight_file != best_checkpoint:
            os.remove(weight_file)
    
    
    plt.figure()
    plt.plot(range(1, len(best_hist.history['loss']) + 1), best_hist.history['loss'], label='train_loss', color='blue')
    plt.plot(range(1, len(best_hist.history['val_loss']) + 1), best_hist.history['val_loss'], label='val_loss', color='orange')
    plt.grid()
    plt.xlabel("Epochs")
    plt.ylabel("Loss")
    plt.legend()
    plt.savefig(os.path.join(OUTPUT_DIR_PATH, f'fold_{fold+1}', 'loss.png'), bbox_inches='tight')
    plt.close()
    
    plt.figure()
    plt.plot(range(1, len(best_hist.history['accuracy']) + 1), best_hist.history['accuracy'], label='train_accuracy', color='blue')
    plt.plot(range(1, len(best_hist.history['val_accuracy']) + 1), best_hist.history['val_accuracy'], label='val_accuracy', color='orange')
    plt.grid()
    plt.xlabel("Epochs")
    plt.ylabel("Accuracy")
    plt.legend()
    plt.savefig(os.path.join(OUTPUT_DIR_PATH, f'fold_{fold+1}', 'accuracy.png'), bbox_inches='tight')
    plt.close()
    
     
    
    fold_accuracies.append(best_test_accuracy)
    
    y_pred = best_model.predict(X_test, batch_size=BATCH_SIZE)
    
    all_probabilities.append(y_pred)
    
    for i, pred in enumerate(np.argmax(y_pred,axis=1)):
        if np.argmax(y_test, axis=1)[i] != pred:
            true_label = emotions_dict_inv[np.argmax(y_test, axis=1)[i]]
            fig, ax = plt.subplots()
            ax.imshow(X_test[i]), plt.title(f'True : {true_label}, Pred : {emotions_dict_inv[pred]}')
            plt.axis('off')
            plt.savefig(os.path.join(OUTPUT_DIR_PATH, f'fold_{fold+1}', 'false_predict', f'test_{i}.png'), bbox_inches='tight')
            plt.close()
    
    
    
    cm = confusion_matrix(np.argmax(y_test, axis=1), np.argmax(y_pred, axis=1))
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=labels)
    disp.plot(cmap=plt.cm.Greens)
    
    plt.savefig(os.path.join(OUTPUT_DIR_PATH, f'fold_{fold+1}', 'cm.png'), bbox_inches='tight')
    plt.close()  
    
    folds = list(range(1, 11))
    
    
    print(cm)
    total_per_class = np.sum(cm, axis=0)
    total_per_class = np.where(total_per_class == 0, 1, total_per_class)
    accuracy_per_class = list((np.diag(cm) / total_per_class)*100)
    print(accuracy_per_class)
    accuracy_per_class.append(np.mean(accuracy_per_class))
    accuracy_per_class.append(np.std(accuracy_per_class[:-1]))
    accuracy_per_fold.append(accuracy_per_class)
    
    with open(os.path.join(OUTPUT_DIR_PATH, f'fold_{fold+1}', 'results.txt'), 'w', encoding='utf-8') as f:
        f.write(f'Test accuracy : {str(test_acc)}\n')
        f.write(f'Probabilities : \n')
        for i, prob in enumerate(y_pred):
            f.write(f'({str(i)}) : {str(prob)}\n')

Fold 1: Train Data Shape: (1932, 100, 100, 3), Train Labels Shape: (1932, 7)
Fold 1: Test Data Shape: (215, 100, 100, 3), Test Labels Shape: (215, 7)

### Essai : 1 / 3 ###
Epoch 1/1000
 9/61 [===>..........................] - ETA: 1:43 - loss: 2.8836 - accuracy: 0.1875

KeyboardInterrupt: 

In [ ]:
with open(os.path.join(OUTPUT_DIR_PATH, 'results.txt'), 'w', encoding='utf-8') as f:
    f.write(f'Mean test accuracy : {str(np.mean(test_acc))}\n')

In [ ]:
accuracy_per_fold.append(np.mean(accuracy_per_fold, axis=0))

row_names = [f"Fold_{i}" for i in range(1, 11)] + ["Mean"]
df = pd.DataFrame(accuracy_per_fold, columns=cols)
df.index = row_names
print(df)
df.to_excel(os.path.join(OUTPUT_DIR_PATH, 'results.xlsx'), index=True)